# Machine Learning Based Recommendation Systems
## Segment 2 - Model-based Collaborative Filtering Systems
## SVD Matrix Factorization

In [29]:
import numpy as np
import pandas as pd

import sklearn
from sklearn.decomposition import TruncatedSVD

The MovieLens dataset was collected by the GroupLens Research Project at the University of Minnesota. You can download the dataset for this demostration at the following URL: https://grouplens.org/datasets/movielens/100k/

### Preparing the data

In [30]:
columns = ['user_id', 'item_id', 'rating', 'timestamp']
frame = pd.read_csv('u.data', sep='\t', names=columns)
frame.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [31]:
columns = ['item_id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
          'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

movies = pd.read_csv('u.item', sep='|', names=columns, encoding='latin-1')
movie_names = movies[['item_id', 'movie title']]
movie_names.head()

,item_id,movie title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [32]:
# Merging data to display movies and rating given by users

combined_movies_data = pd.merge(frame, movie_names, on='item_id')
combined_movies_data.head()

,user_id,item_id,rating,timestamp,movie title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [33]:
# Count of ratings received by different movies based on item id

combined_movies_data.groupby('item_id')['rating'].count().sort_values(ascending=False).head()

item_id
50     583
258    509
100    508
181    507
294    485
Name: rating, dtype: int64

In [34]:
# title of most reviewed movie 

filter = combined_movies_data['item_id']==50
combined_movies_data[filter]['movie title'].unique()  # Return name only once

array([u'Star Wars (1977)'], dtype=object)

### Building a Utility Matrix

User data of reviews for all movies, this matrix will be sparce as only few movies are reviewed/rated by the users


In [35]:
# Creating utility matrix, if no values or Null, replacing it with 0

rating_crosstab = combined_movies_data.pivot_table(values='rating', index='user_id', columns='movie title', fill_value=0)
rating_crosstab.head()

movie title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,2,5,0,0,3,4,0,0,...,0,0,0,5,3,0,0,0,4,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,2,0,0,0,0,4,0,0,...,0,0,0,4,0,0,0,0,4,0


### Transposing the Matrix

Transposing utility matrix to review movie against user ratings, before we pass it to Truncated SVD

In [36]:
rating_crosstab.shape

(943, 1664)

In [37]:
X = rating_crosstab.T
X.shape

(1664, 943)

### Decomposing the Matrix

In [38]:
# Truncated SVD is used to generalize the view of user's interest in moviews

# Reducing dimension to 12 components

SVD = TruncatedSVD(n_components=12, random_state=17)

resultant_matrix = SVD.fit_transform(X)

resultant_matrix.shape

(1664, 12)

### Generating a Correlation Matrix

In [39]:
corr_mat = np.corrcoef(resultant_matrix)    # Pearson R correlation
corr_mat.shape

(1664, 1664)

### Isolating Star Wars From the Correlation Matrix

Using ***'Star Wars 1977'*** as the movie of interest

In [40]:
movie_names = rating_crosstab.columns
movies_list = list(movie_names)

star_wars = movies_list.index('Star Wars (1977)')
star_wars

1398

In [41]:
corr_star_wars = corr_mat[1398]
corr_star_wars.shape


(1664,)

### Recommending a Highly Correlated Movie

Movies highly recommended to the users who liked ***Star wars 1977***

In [52]:
list(movie_names[(corr_star_wars <= 0.99) & (corr_star_wars > 0.9)])


[u'Die Hard (1988)',
 u'Empire Strikes Back, The (1980)',
 u'Fugitive, The (1993)',
 u'Raiders of the Lost Ark (1981)',
 u'Return of the Jedi (1983)',
 u'Terminator 2: Judgment Day (1991)',
 u'Terminator, The (1984)',
 u'Toy Story (1995)']

In [51]:
list(movie_names[(corr_star_wars <= 0.99) & (corr_star_wars > 0.95)])

[u'Return of the Jedi (1983)']